In [ ]:
# default_exp entry_point

In [ ]:
#export
import os
import argparse
import time
import importlib
import inspect

def main():
    print('in gcp_runner entry point')
    args_parser = argparse.ArgumentParser()
    args_parser.add_argument(
        '--module-name',
        help='module name of an app to run',
        required=True)
    args_parser.add_argument(
        '--function-name',
        help='function name to run',
        required=True)
    
    args, unknown_args = args_parser.parse_known_args()
    print('running entrypoint function: %s.%s' % (args.module_name, args.function_name))
    module = importlib.import_module(args.module_name)
    func = getattr(module, args.function_name)
    if unknown_args is not None and len(unknown_args) > 0:
        print('additional args: %s' % str(unknown_args))
        args_spec = inspect.getargspec(func)
        if args_spec is not None and (len(args_spec.args) == 1 or args_spec.varargs == 'args'):
            func(unknown_args)
            return
    
    func()
    
if __name__ == '__main__':
    main()

in gcp_runner entry point


usage: ipykernel_launcher.py [-h] --module-name MODULE_NAME --function-name
                             FUNCTION_NAME
ipykernel_launcher.py: error: argument --module-name is required


SystemExit: 2

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted entry_point.ipynb.
Converted index.ipynb.
Converted run_local.ipynb.
Converted sample_code.ipynb.
